In [30]:
import numpy
import numpy as np
import math
import plotly.graph_objects as go
import plotly.express as px

In [3]:
def piksel(koord):
    return np.array([1,-1,-1])*(np.array([1600,0,0])-koord) #promeni 1600 za sliku

In [4]:
def fundJed(xl,xd):
    return np.array([xl[0]*xd[0],xl[1]*xd[0],xl[2]*xd[0],xl[0]*xd[1],xl[1]*xd[1],xl[2]*xd[1],xl[0]*xd[2],xl[1]*xd[2],xl[2]*xd[2]])

In [5]:
def fundamentalnaMatrica(leve,desne):
    a=[]
    for x,xp in zip(leve,desne):
        jed= fundJed(x,xp)
        a.append(jed)
    a = np.array(a)
    U, D, V = np.linalg.svd(a)
    f = V[-1]
    f = f.reshape(3,3)
    return f

In [69]:
def skew2v(EC):
    return np.array([EC[2][1],EC[0][2],EC[1][0]])

In [148]:
def dekomponuj(e):
    q0=np.array([[0,-1,0],[1,0,0],[0,0,1]])
    e0=np.array([[0,1,0],[-1,0,0],[0,0,0]])
    U,S,V=np.linalg.svd(e)
    EC=U@e0@np.transpose(U)
    A=U@q0@V #probaj verziju bez transponovanih q0,v ako nesto ne valja
    return A,skew2v(EC) #skew2v mozda vrati suprotan znak, stavi mu minus ako ne valja

In [163]:
def kamera1(e):
    A,C=dekomponuj(e)
    C=K1 @ -np.transpose(A) @ C
    A=K1 @ np.transpose(A)
    A1=np.concatenate((np.transpose(A),[C]),axis=0) #append A.T,C
    return np.transpose(A1)

In [164]:
def triangulisi(T1,T2,pt1,pt2):
    a=np.array([pt1[1]*T1[2,:]-pt1[2]*T1[1,:],
                -pt1[0]*T1[2,:]+pt1[2]*T1[0,:],
                pt2[1]*T2[2,:]-pt2[2]*T2[1,:],
                -pt2[0]*T2[2,:]+pt2[2]*T2[0,:]]) #ima na slajdovima
    U,S,V=np.linalg.svd(a)
    pt=V[3] # valjda moze -1
    pt=(1/pt[3])*pt
    pt=np.where(pt==0,0.0,pt)
    return np.array(pt)

In [161]:
def recPts(c1,c2,pts1,pts2):
    a=[]
    for x,xp in zip(pts1,pts2):
        X=triangulisi(c1,c2,x,xp)
        X=np.delete(X,-1)
        a.append(X)
    a = np.array(a)
    return a

In [162]:
K1=np.array([[1300,0,800],[0,1300,600],[0,0,1]])
kam2=np.array([[1300,0,800,0],
             [0,1600,600,0],
             [0,0,1,0]])
tacke1=np.array([[383, 264, 1],
                [727, 351, 1],
                [551, 651, 1],
                [147, 524, 1],
                [1228, 971, 1],
                [1556, 757, 1],
                [1143, 587, 1], 
                [1456, 508, 1]])
tacke2=np.array([[489, 183, 1],
                [731, 307, 1],
                [467, 514, 1],
                [214, 347, 1],
                [1050, 994, 1],
                [1491, 903, 1],
                [1127, 630, 1],
                [1343, 589, 1]])
all1p=np.array([[454, 507, 1],
                [740, 596, 1],
                [601, 870, 1],
                [283, 761, 1],
                [427, 371, 1],
                [825, 403, 1],
                [1162, 410,1],
                [1167, 251,1],
                [828, 247,1],
                [1102, 323,1],
                [919, 322,1]])
all2p=np.array([[537, 420, 1],
                [744, 552, 1],
                [530, 746, 1],
                [317, 857, 1],
                [479, 274, 1],
                [897, 404, 1],
                [1223, 482,1],
                [1263, 322,1],
                [933, 257,1],
                [1167, 377,1],
                [988, 337,1]])
allL=np.concatenate((tacke1,all1p),axis=0)
allD=np.concatenate((tacke2,all2p),axis=0)
#fundamentalna matrica
f=fundamentalnaMatrica(piksel(tacke1),piksel(tacke2))
#osnovna matrica
e=np.transpose(K1)@f@K1
pts3D=recPts(kamera1(e),kam2,allL,allD)
pts3D

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3 and the array at index 1 has size 1

In [158]:
import plotly.graph_objects as go
import plotly.express as px

# U ovom slucaju crtamo jednostavnu kocku. Vi cete koristiti rekonstruisana temena
temenaKocke = pts3D #np.array([[0,0, 3], [0, 3,3], [3,3,3],[3, 0,3],[0,0,0], [0, 3,0], [3, 3,0],[3,0,0]] )
# Ovo su ivice kocke zadate indeksima temena
ivice = [[0, 1], [0, 3], [0, 4], [2, 3], [2, 1], [2, 6], [5, 6], [5, 4], [5, 1], [7, 6],
[7, 3], [7, 4], [9, 10], [9, 12], [9, 14], [11, 12], [11, 10], [11, 13], [13, 14],
[10, 13], [14, 12], [15, 16], [15, 17], [15, 18], [16, 17], [16, 18], [17, 18]]

# dodatne mogucnosti istrazite sami

#  prikaz scene - koristi "plotly" biblioteku
def prikazKocke(): 
    # izdvajamo x,y,z koordinate svih tacaka
    xdata = (np.transpose(temenaKocke))[0]
    ydata = (np.transpose(temenaKocke))[1]
    zdata = (np.transpose(temenaKocke))[2]
    # u data1 ubacujemo sve sto treba naccrtati
    data1 = []
    # za svaku ivicu crtamo duz na osnovu koordinata
    for i in range(len(ivice)):
        data1.append(go.Scatter3d(x=[xdata[ivice[i][0]], xdata[ivice[i][1]]], y=[ydata[ivice[i][0]], ydata[ivice[i][1]]],z=[zdata[ivice[i][0]], zdata[ivice[i][1]]]))
    fig = go.Figure(data = data1 )
    # da ne prikazuje legendu
    fig.update_layout(showlegend=False)
    # pravi html fajl (ako zelite da napravite "rotatable" 3D rekonstruciju)
    # birate kao parametar velicinu apleta. fulhtml=False je vazno da ne bi pravio ogroman fajl
    # ovde stavite neki vas folder
    fig.write_html("./rekonstrukcija", include_plotlyjs = 'cdn', default_width = '800px', default_height = '600px', full_html = False) #Modifiy the html file
    fig.show()

prikazKocke()